In [1]:
import pandas as pd

# Laden der Excel-Datei
base_path = 'C:/Users/paulu/OneDrive/Desktop/Masterarbeit/data'
file_path = base_path + '/colon_cancer_1998to2003.xlsx'
data = pd.read_excel(file_path)

# Anzeigen der ersten paar Zeilen, um einen Überblick über die Struktur der Daten zu bekommen
data.head()


,EOD 10 - extent (1988-2003),SEER modified AJCC stage 3rd (1988-2003),Regional nodes positive (1988+),RX Summ--Surg Prim Site (1998+),Histologic Type ICD-O-3,Age recode with single ages and 90+,Diagnostic Confirmation,EOD Primary Tumor (2018+),Behavior code ICD-O-3,Primary Site,Survival months,SEER cause-specific death classification
0,45,30,3,30,8480,55 years,Positive histology,Blank(s),Malignant,186,173,Alive or dead of other cause
1,85,40,1,30,8140,72 years,Positive histology,Blank(s),Malignant,185,22,Dead (attributable to this cancer dx)
2,45,30,97,30,8140,55 years,Positive histology,Blank(s),Malignant,187,26,Dead (attributable to this cancer dx)
3,85,40,98,0,8140,71 years,Positive histology,Blank(s),Malignant,187,3,Dead (attributable to this cancer dx)
4,45,20,0,40,8140,47 years,Positive histology,Blank(s),Malignant,186,223,Alive or dead of other cause


In [2]:
data.describe()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49533 entries, 0 to 49532
Data columns (total 12 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   EOD 10 - extent (1988-2003)               49533 non-null  int64 
 1   SEER modified AJCC stage 3rd (1988-2003)  49533 non-null  object
 2   Regional nodes positive (1988+)           49533 non-null  int64 
 3   RX Summ--Surg Prim Site (1998+)           49533 non-null  int64 
 4   Histologic Type ICD-O-3                   49533 non-null  int64 
 5   Age recode with single ages and 90+       49533 non-null  object
 6   Diagnostic Confirmation                   49533 non-null  object
 7   EOD Primary Tumor (2018+)                 49533 non-null  object
 8   Behavior code ICD-O-3                     49533 non-null  object
 9   Primary Site                              49533 non-null  int64 
 10  Survival months                           4953

In [3]:
# Erstellen einer Liste mit den Ausprägungen pro Feature (Spalte)
feature_values = {col: data[col].unique() for col in data.columns}

# Anzeigen der Ausprägungen für jedes Feature
feature_values


{'EOD 10 - extent (1988-2003)': array([45, 85, 40, 11, 16, 15, 20, 99, 60, 55, 65, 75, 10, 50, 70, 14, 30,
        42, 12, 80, 13, 66]),
 'SEER modified AJCC stage 3rd (1988-2003)': array([30, 40, 20, 10, 90, 'Stage 0', 99, 88], dtype=object),
 'Regional nodes positive (1988+)': array([ 3,  1, 97, 98,  0,  6,  2,  8, 14, 10,  4,  5, 16, 12,  9,  7, 80,
        18, 99, 13, 19, 26, 11, 15, 21, 25, 22, 17, 23, 20, 24, 35, 29, 48,
        32, 31, 30, 33, 38, 42, 46, 36, 67, 64, 27, 58, 28, 69, 44, 39, 34]),
 'RX Summ--Surg Prim Site (1998+)': array([30,  0, 40, 28, 26, 70, 50, 27, 60, 20, 32, 80, 90, 41, 99, 22, 24,
        29, 25, 51, 12, 23, 14, 10, 61]),
 'Histologic Type ICD-O-3': array([8480, 8140, 8263, 8490, 8210, 8261, 8481, 8240, 8010, 8262, 8000,
        8211, 8255, 8243, 8550, 8246, 8070, 8720, 8021, 8123, 8470, 8936,
        8460, 8020, 8940, 8245, 8221, 8560, 8890, 8260, 8032, 8574, 8220,
        8570, 8244, 8001, 8801, 8145, 8046, 8310, 8013, 8510, 8050, 8041,
        8141, 8

In [4]:
# Eliminiere Zeilen mit "Unknown" oder "NV" in der 'Survival months'-Spalte
data = data[~data['Survival months'].isin(['Unknown', 'NV'])]

# Umwandlung der 'Survival months'-Spalte in numerische Werte (falls notwendig)
data['Survival months'] = pd.to_numeric(data['Survival months'], errors='coerce')

# Erstellen der binären Überlebensvariable
data['Survival Status'] = 0  # Standardwert ist 'tod'
data.loc[(data['Survival months'] >= 60) | (data['SEER cause-specific death classification'] != 'Dead (attributable to this cancer dx)'), 'Survival Status'] = 1

# Anzeigen der ersten Zeilen, um die neue Variable zu überprüfen
data[['Survival months', 'SEER cause-specific death classification', 'Survival Status']].head()


,Survival months,SEER cause-specific death classification,Survival Status
0,173,Alive or dead of other cause,1
1,22,Dead (attributable to this cancer dx),0
2,26,Dead (attributable to this cancer dx),0
3,3,Dead (attributable to this cancer dx),0
4,223,Alive or dead of other cause,1


In [18]:
total_patients = len(data)
count_age90plus = data[data['Age recode with single ages and 90+'] == "90+ years"].shape[0]
percentage_age90plus = (count_age90plus / total_patients) * 100
percentage_age90plus

#count_age90plus = data[data['Age recode with single ages and 90+'] == "90+ years"].shape[0]
#count_age90plus

4.584269662921348

In [19]:
# Entferne alle Zeilen mit "90+Years"
data = data[~data['Age recode with single ages and 90+'].str.startswith('90')]

# Extrahiere die numerischen Alterswerte und konvertiere sie zu Integer
data['Age recode with single ages and 90+'] = (
    data['Age recode with single ages and 90+']
    .str.extract(r'(\d+)')        # Extrahiere die Zahl
    .astype(int)                  # Wandle sie in Integer um
)

# Optional: Spalte umbenennen (z. B. in "Age")
data = data.rename(columns={'Age recode with single ages and 90+': 'Age'})


C:\Users\paulu\AppData\Local\Temp\ipykernel_16104\4220121157.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Age recode with single ages and 90+'] = (


In [20]:
# Erstellen einer Liste mit den Ausprägungen pro Feature (Spalte)

feature_values = data['Age'].unique()

# Anzeigen der Ausprägungen für jedes Feature
feature_values.sort()
feature_values


array([ 4,  9, 11, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
       28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44,
       45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61,
       62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78,
       79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

for col in data.select_dtypes(include=['number']).columns:
    plt.figure()
    plt.hist(data[col].dropna(),bins=30)
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel('Häufigkeit')
    plt.grid(True)
    plt.show()